# DATA 607 -- Assignment 3

In this assignment, we apply the ideas underlying dense word embeddings like Word2Vec and GloVe to construct dense embeddings of categorical features.

The context of our exploration will be the [Rossmann Store Sales Competition](https://www.kaggle.com/c/rossmann-store-sales/overview/description) from *Kaggle*, the goal of which is to forecast store sales using store, promotion, and competitor data.

## Instructions

1. Download the data from the competition page or from [my github](https://github.com/mgreenbe/rossmann).

2. Replace each date in the `Date` column with number of days between it and January 1, 2013, the earliest date in the table.

3. Use `pd.get_dummies` to construct dataframes `stores`, `days_of_week`, and `state_holidays` containing 1-hot encodings of the categorical variables `Store`, `DayOfWeek`, and `StateHoliday`, respectively.

4. Assemble these encoded features, together with the numerical ones (`Date`, `Customers`) and binary ones (`Open`, `Promo`, `SchoolHoliday`), in a matrix `X`, the first 1115 columns of which represent the store ID.

5. Split the data `X` and `Y` into training and validation sets. Standardize the numerical feature columns. Here, the relevant means and standard deviations should be computed from *training data*.

6. Train the model `MyModel`, below, using `MeanSquaredLogarithmicError` as the loss function. Explain, briefly, why this is an appropriate choice of loss function. Stop training when validation error stabilizes.

7. **(Optional)** Add hidden layers to this model and tune the `store_emb_dim` hyperparameter to improve your results.


In [85]:
from tensorflow import keras

class MyModel(keras.Model):
  def __init__(self, n_stores=1115, store_emb_dim=20):
    super(MyModel, self).__init__()
    self.n_stores = n_stores
    self.encoder = keras.layers.Dense(store_emb_dim, name="encoder")
    self.regressor = keras.layers.Dense(1, name="regressor")

  def call(self, X):
    x = tf.concat([self.encoder(X[:, :self.n_stores]), X[:, self.n_stores:]], axis=-1)
    return self.regressor(x)

- This is model is *not* built with `keras.models.Sequential` -- it's not simply passing data through a sequence of layers. The first 1115 columns of the input, representing the store ID, are projected onto a `store_emb_dim`-dimensional space. The resulting projections are then concatenated with the remaining features before applying linear regression. (Notice the absence of nonlinear activation functions.)

- **Warning:** The data set contains > 1 million rows. To avoid running out of memory, work initially with a subset of the rows (say, 10,000). Train on as large a subset of the whole dataset as you can without crashing your session.

## References

Rachel Thomas, [An Introduction to Deep Learning for Tabular Data](https://www.fast.ai/2018/04/29/categorical-embeddings/) (fast.ai blog, April 29, 2018)

Cheng Guo and Felix Berkhahn, [Entity Embeddings of Categorical Variables](https://arxiv.org/pdf/1604.06737.pdf) (April 25, 2016)

In [86]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import *

## Q1. 

In [87]:
test_url='https://raw.githubusercontent.com/mgreenbe/rossmann/main/test.csv'
train_url='https://raw.githubusercontent.com/mgreenbe/rossmann/main/train.csv'

In [88]:
df_test = pd.read_csv(test_url,encoding="latin-1")
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [89]:
df_test.shape

(41088, 8)

In [90]:
df_train = pd.read_csv(train_url,encoding="latin-1",dtype=object)
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [91]:
df_train.shape

(1017209, 9)

## Q2.

In [92]:
#changing the date format to what the actual data is like
date_format = "%Y-%m-%d"
#earliest date 
earliest_date = datetime.strptime('2013-01-01', date_format)

for rows in range(len(df_train)):
    #obtaining the date at current row of traning dataframe
    current_date=datetime.strptime(df_train.iat[rows,2], date_format)
    #replacing date
    delta=current_date - earliest_date
    df_train.iat[rows,2]= delta.days

In [93]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,941,5263,555,1,1,0,1
1,2,5,941,6064,625,1,1,0,1
2,3,5,941,8314,821,1,1,0,1
3,4,5,941,13995,1498,1,1,0,1
4,5,5,941,4822,559,1,1,0,1


In [94]:
#same logic as above but for test data
for rows in range(len(df_test)):
    #obtaining the date at current row of traning dataframe
    current_date=datetime.strptime(str(df_test.iat[rows,3]), date_format)
    #replacing date
    delta=current_date - earliest_date
    df_test.iat[rows,3]= delta.days


In [95]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,989,1.0,1,0,0
1,2,3,4,989,1.0,1,0,0
2,3,7,4,989,1.0,1,0,0
3,4,8,4,989,1.0,1,0,0
4,5,9,4,989,1.0,1,0,0


## Q3.

In [96]:
#for training data
store_train =pd.get_dummies(df_train.Store)
days_of_week_train =pd.get_dummies(df_train.DayOfWeek)
state_holidays_train=pd.get_dummies(df_train.StateHoliday)

In [97]:
len(df_train.Store.unique())

1115

In [98]:
store_train.shape

(1017209, 1115)

In [99]:
store_train.head(1)

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,990,991,992,993,994,995,996,997,998,999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
days_of_week_train.head(1)

,1,2,3,4,5,6,7
0,0,0,0,0,1,0,0


In [101]:
days_of_week_train.shape

(1017209, 7)

It can be observed that the number of columns in store_train dataframe are the unique entries of the Store column in df_train dataset

In [102]:
#for test data
store_test =pd.get_dummies(df_test.Store)
days_of_week_test =pd.get_dummies(df_test.DayOfWeek)
state_holidays_test=pd.get_dummies(df_test.StateHoliday)

## Q4.

In [103]:
#train
X=pd.concat([store_train, days_of_week_train,state_holidays_train,df_train.Date,df_train.Sales,df_train.Customers,df_train.Open,
            df_train.Promo,df_train.SchoolHoliday], axis=1)

In [104]:
X.shape

(1017209, 1132)

In [105]:
#test
Y=pd.concat([store_test, days_of_week_test,state_holidays_test,df_test.Date,df_test.Open,
            df_test.Promo,df_test.SchoolHoliday], axis=1)

In [106]:
Y.shape

(41088, 869)

## Q5.

Since the test data does not have the Sales variable I will not use it.I will instead explcitly state Y is sales. Further, I will drop the Sales column from the X dataframe.

In [107]:
Y=X.Sales

In [108]:
X.drop(columns=['Sales'],inplace=True)

In [144]:
X_train,X_val,Y_train,Y_val=train_test_split(X, Y, test_size=0.2, random_state=42)

**Standadizing numerical feature columns(Date ,Customers for X)
with means and standard deviation obtaining in the training data**

In [145]:
X_train.shape, X_val.shape, Y_train.shape, Y_val.shape

((813767, 1131), (203442, 1131), (813767,), (203442,))

In [146]:
scaler_Date = StandardScaler().fit(df_train.Date.values.reshape(-1,1))
scaler_Customer = StandardScaler().fit(df_train.Customers.values.reshape(-1,1))

In [138]:
print(scaler_Date.mean_,scaler_Customer.mean_)

[465.0629959] [633.14594641]


**Standardize data**

In [139]:
X_train.head(1)

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,7,0,a,b,c,Date,Customers,Open,Promo,SchoolHoliday
417683,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,537,0,0,0,0


In [140]:
X_val.head(1)

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,7,0,a,b,c,Date,Customers,Open,Promo,SchoolHoliday
76435,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,873,0,0,0,0


**Standardize Date**

In [147]:
X_train.Date=scaler_Date.transform(X_train.Date.values.reshape(-1,1))
X_val.Date=scaler_Date.transform(X_val.Date.values.reshape(-1,1))

C:\Users\Edwin Aguirre\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\Edwin Aguirre\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


**Standardize Customers**

In [148]:
X_train.Customers=scaler_Customer.transform(X_train.Customers.values.reshape(-1,1))
X_val.Customers=scaler_Customer.transform(X_val.Customers.values.reshape(-1,1))

C:\Users\Edwin Aguirre\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\Edwin Aguirre\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [149]:
X_train1.head(1)

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,7,0,a,b,c,Date,Customers,Open,Promo,SchoolHoliday
459018,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0.127296,0.197786,1,0,0


In [150]:
X_val1.head(1)

,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,7,0,a,b,c,Date,Customers,Open,Promo,SchoolHoliday
816535,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,-1.042299,-1.36333,1,0,0


## Q6.

**Model Defined**

In [151]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding,GRU
class MyModel(keras.Model):
    def __init__(self, n_stores=1115, store_emb_dim=20):
        super(MyModel, self).__init__()
        self.n_stores = n_stores
        self.encoder = keras.layers.Dense(store_emb_dim, name="encoder")
        self.regressor = keras.layers.Dense(1, name="regressor")

    def call(self, X):
        x = tf.concat([self.encoder(X[:, :self.n_stores]), X[:, self.n_stores:]], axis=-1)
        return self.regressor(x)

**Training model**

#### Random sampling of 20,000 to skip out of memory issue

In [155]:
X_validamall=X_val.sample(n=20000, random_state=42)
Y_validamall=Y_val[X_validamall.index]

In [156]:
X_trainsmall=X_train.sample(n=20000, random_state=42)
Y_trainsmall=Y_train[X_trainsmall.index]
#Adding callbacks
cb = EarlyStopping(monitor='val_mean_squared_logarithmic_error', mode='auto', verbose=1,patience=10)  

In [157]:
X_trainsmall.shape, Y_trainsmall.shape, X_validamall.shape,Y_validamall.shape

((20000, 1131), (20000,), (20000, 1131), (20000,))

In [158]:
model=MyModel()
model.compile(optimizer='adam',loss="MeanSquaredLogarithmicError",metrics=["MeanSquaredLogarithmicError"])
model.fit(x=np.array(X_trainsmall,dtype=np.float16),
          y=np.array(Y_trainsmall,dtype=np.float16),
          validation_data=(np.array(X_validamall,dtype=np.float16),np.array(Y_validamall,dtype=np.float16)),
          epochs=10000,callbacks=[cb])

Epoch 1/10000
625/625 [==============================] - 2s 2ms/step - loss: 51.5471 - mean_squared_logarithmic_error: 51.5471 - val_loss: 33.7653 - val_mean_squared_logarithmic_error: 33.7653
Epoch 2/10000
625/625 [==============================] - 1s 1ms/step - loss: 31.5602 - mean_squared_logarithmic_error: 31.5602 - val_loss: 26.6889 - val_mean_squared_logarithmic_error: 26.6889
Epoch 3/10000
625/625 [==============================] - 1s 1ms/step - loss: 25.5834 - mean_squared_logarithmic_error: 25.5834 - val_loss: 22.8969 - val_mean_squared_logarithmic_error: 22.8969
Epoch 4/10000
625/625 [==============================] - 1s 1ms/step - loss: 22.2294 - mean_squared_logarithmic_error: 22.2294 - val_loss: 20.3830 - val_mean_squared_logarithmic_error: 20.3830
Epoch 5/10000
625/625 [==============================] - 1s 1ms/step - loss: 19.8651 - mean_squared_logarithmic_error: 19.8651 - val_loss: 18.5408 - val_mean_squared_logarithmic_error: 18.5408
Epoch 6/10000
625/625 [============

625/625 [==============================] - 1s 1ms/step - loss: 9.4465 - mean_squared_logarithmic_error: 9.4465 - val_loss: 10.6819 - val_mean_squared_logarithmic_error: 10.6819
Epoch 86/10000
625/625 [==============================] - 1s 2ms/step - loss: 9.4985 - mean_squared_logarithmic_error: 9.4985 - val_loss: 10.6711 - val_mean_squared_logarithmic_error: 10.6711
Epoch 87/10000
625/625 [==============================] - 1s 2ms/step - loss: 9.2967 - mean_squared_logarithmic_error: 9.2967 - val_loss: 10.6482 - val_mean_squared_logarithmic_error: 10.6482
Epoch 88/10000
625/625 [==============================] - 1s 2ms/step - loss: 9.4257 - mean_squared_logarithmic_error: 9.4257 - val_loss: 10.6388 - val_mean_squared_logarithmic_error: 10.6388
Epoch 89/10000
625/625 [==============================] - 1s 2ms/step - loss: 9.3295 - mean_squared_logarithmic_error: 9.3295 - val_loss: 10.6214 - val_mean_squared_logarithmic_error: 10.6214
Epoch 90/10000
625/625 [==============================]

Epoch 170/10000
625/625 [==============================] - 1s 2ms/step - loss: 6.9634 - mean_squared_logarithmic_error: 6.9634 - val_loss: 9.2954 - val_mean_squared_logarithmic_error: 9.2954
Epoch 171/10000
625/625 [==============================] - 1s 2ms/step - loss: 7.0378 - mean_squared_logarithmic_error: 7.0378 - val_loss: 9.2807 - val_mean_squared_logarithmic_error: 9.2807
Epoch 172/10000
625/625 [==============================] - 1s 2ms/step - loss: 7.0649 - mean_squared_logarithmic_error: 7.0649 - val_loss: 9.2654 - val_mean_squared_logarithmic_error: 9.2654
Epoch 173/10000
625/625 [==============================] - 1s 2ms/step - loss: 7.0434 - mean_squared_logarithmic_error: 7.0434 - val_loss: 9.2534 - val_mean_squared_logarithmic_error: 9.2534
Epoch 174/10000
625/625 [==============================] - 1s 2ms/step - loss: 7.0092 - mean_squared_logarithmic_error: 7.0092 - val_loss: 9.2386 - val_mean_squared_logarithmic_error: 9.2386
Epoch 175/10000
625/625 [====================

Epoch 213/10000
625/625 [==============================] - 1s 2ms/step - loss: 6.2644 - mean_squared_logarithmic_error: 6.2644 - val_loss: 8.6366 - val_mean_squared_logarithmic_error: 8.6366
Epoch 214/10000
625/625 [==============================] - 1s 2ms/step - loss: 6.3536 - mean_squared_logarithmic_error: 6.3536 - val_loss: 8.6214 - val_mean_squared_logarithmic_error: 8.6214
Epoch 215/10000
625/625 [==============================] - 1s 2ms/step - loss: 6.1721 - mean_squared_logarithmic_error: 6.1721 - val_loss: 8.6092 - val_mean_squared_logarithmic_error: 8.6092
Epoch 216/10000
625/625 [==============================] - 1s 2ms/step - loss: 6.2376 - mean_squared_logarithmic_error: 6.2376 - val_loss: 8.5911 - val_mean_squared_logarithmic_error: 8.5911
Epoch 217/10000
625/625 [==============================] - 1s 1ms/step - loss: 6.0666 - mean_squared_logarithmic_error: 6.0666 - val_loss: 8.5769 - val_mean_squared_logarithmic_error: 8.5769
Epoch 218/10000
625/625 [====================

Epoch 256/10000
625/625 [==============================] - 1s 2ms/step - loss: 5.6556 - mean_squared_logarithmic_error: 5.6556 - val_loss: 7.9921 - val_mean_squared_logarithmic_error: 7.9921
Epoch 257/10000
625/625 [==============================] - 1s 2ms/step - loss: 5.6566 - mean_squared_logarithmic_error: 5.6566 - val_loss: 7.9758 - val_mean_squared_logarithmic_error: 7.9758
Epoch 258/10000
625/625 [==============================] - 1s 1ms/step - loss: 5.6265 - mean_squared_logarithmic_error: 5.6265 - val_loss: 7.9628 - val_mean_squared_logarithmic_error: 7.9628
Epoch 259/10000
625/625 [==============================] - 1s 2ms/step - loss: 5.6016 - mean_squared_logarithmic_error: 5.6016 - val_loss: 7.9464 - val_mean_squared_logarithmic_error: 7.9464
Epoch 260/10000
625/625 [==============================] - 1s 1ms/step - loss: 5.6864 - mean_squared_logarithmic_error: 5.6864 - val_loss: 7.9367 - val_mean_squared_logarithmic_error: 7.9367
Epoch 261/10000
625/625 [====================

...

Epoch 1503/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.1230 - mean_squared_logarithmic_error: 1.1230 - val_loss: 2.0964 - val_mean_squared_logarithmic_error: 2.0964
Epoch 1504/10000
625/625 [==============================] - 1s 2ms/step - loss: 1.1278 - mean_squared_logarithmic_error: 1.1278 - val_loss: 2.0952 - val_mean_squared_logarithmic_error: 2.0952
Epoch 1505/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.1275 - mean_squared_logarithmic_error: 1.1275 - val_loss: 2.0970 - val_mean_squared_logarithmic_error: 2.0970
Epoch 1506/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.1374 - mean_squared_logarithmic_error: 1.1374 - val_loss: 2.1032 - val_mean_squared_logarithmic_error: 2.1032
Epoch 1507/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.1269 - mean_squared_logarithmic_error: 1.1269 - val_loss: 2.0900 - val_mean_squared_logarithmic_error: 2.0900
Epoch 1508/10000
625/625 [==============

Epoch 1546/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0904 - mean_squared_logarithmic_error: 1.0904 - val_loss: 2.0512 - val_mean_squared_logarithmic_error: 2.0512
Epoch 1547/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0888 - mean_squared_logarithmic_error: 1.0888 - val_loss: 2.0565 - val_mean_squared_logarithmic_error: 2.0565
Epoch 1548/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0942 - mean_squared_logarithmic_error: 1.0942 - val_loss: 2.0513 - val_mean_squared_logarithmic_error: 2.0513
Epoch 1549/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0908 - mean_squared_logarithmic_error: 1.0908 - val_loss: 2.0537 - val_mean_squared_logarithmic_error: 2.0537
Epoch 1550/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0861 - mean_squared_logarithmic_error: 1.0861 - val_loss: 2.0540 - val_mean_squared_logarithmic_error: 2.0540
Epoch 1551/10000
625/625 [==============

Epoch 1589/10000
625/625 [==============================] - 1s 2ms/step - loss: 1.0546 - mean_squared_logarithmic_error: 1.0546 - val_loss: 2.0213 - val_mean_squared_logarithmic_error: 2.0213
Epoch 1590/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0543 - mean_squared_logarithmic_error: 1.0543 - val_loss: 2.0168 - val_mean_squared_logarithmic_error: 2.0168
Epoch 1591/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0491 - mean_squared_logarithmic_error: 1.0491 - val_loss: 2.0194 - val_mean_squared_logarithmic_error: 2.0194
Epoch 1592/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0481 - mean_squared_logarithmic_error: 1.0481 - val_loss: 2.0135 - val_mean_squared_logarithmic_error: 2.0135
Epoch 1593/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0488 - mean_squared_logarithmic_error: 1.0488 - val_loss: 2.0164 - val_mean_squared_logarithmic_error: 2.0164
Epoch 1594/10000
625/625 [==============

Epoch 1632/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0202 - mean_squared_logarithmic_error: 1.0202 - val_loss: 1.9807 - val_mean_squared_logarithmic_error: 1.9807
Epoch 1633/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0105 - mean_squared_logarithmic_error: 1.0105 - val_loss: 1.9769 - val_mean_squared_logarithmic_error: 1.9769
Epoch 1634/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0098 - mean_squared_logarithmic_error: 1.0098 - val_loss: 1.9704 - val_mean_squared_logarithmic_error: 1.9704
Epoch 1635/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0139 - mean_squared_logarithmic_error: 1.0139 - val_loss: 1.9812 - val_mean_squared_logarithmic_error: 1.9812
Epoch 1636/10000
625/625 [==============================] - 1s 1ms/step - loss: 1.0137 - mean_squared_logarithmic_error: 1.0137 - val_loss: 1.9776 - val_mean_squared_logarithmic_error: 1.9776
Epoch 1637/10000
625/625 [==============

Epoch 1675/10000
625/625 [==============================] - 1s 1ms/step - loss: 0.9888 - mean_squared_logarithmic_error: 0.9888 - val_loss: 1.9435 - val_mean_squared_logarithmic_error: 1.9435
Epoch 1676/10000
625/625 [==============================] - 1s 1ms/step - loss: 0.9930 - mean_squared_logarithmic_error: 0.9930 - val_loss: 1.9407 - val_mean_squared_logarithmic_error: 1.9407
Epoch 1677/10000
625/625 [==============================] - 1s 1ms/step - loss: 0.9969 - mean_squared_logarithmic_error: 0.9969 - val_loss: 1.9381 - val_mean_squared_logarithmic_error: 1.9381
Epoch 1678/10000
625/625 [==============================] - 1s 1ms/step - loss: 0.9822 - mean_squared_logarithmic_error: 0.9822 - val_loss: 1.9409 - val_mean_squared_logarithmic_error: 1.9409
Epoch 1679/10000
625/625 [==============================] - 1s 1ms/step - loss: 0.9835 - mean_squared_logarithmic_error: 0.9835 - val_loss: 1.9360 - val_mean_squared_logarithmic_error: 1.9360
Epoch 1680/10000
625/625 [==============

After 1682 epochs, the validation error stabilized. Therefore, the training of the model "MyModel" stopped. 

**Why is using MSLE a good idea?**

In [173]:
scaler_Y = StandardScaler().fit(Y.values.reshape(-1,1))

In [174]:
print(scaler_Y.mean_,scaler_Y.var_)

[5773.81897231] [14821916.98358157]


Using MSLE is a good idea when the predictor variable has large deviations (high variance),and you don’t want large errors to be significantly more penalized than small ones.Since, the variance of our predictor is quite large(~14821917), using the MSLE is a good idea.